1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 쪼갠 문서를 임베딩해서 벡터 데이터베이스에 저장한다
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색을 한다
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달한다

# 0. RAG 과정을 하기 위해 필요한 도구 설치
- docx2txt : .docx(워드 파일)을 순수 텍스트로 변환해주는 라이브러리이다. 문서 읽기에 필요하다.
- langchain-community : LangChain의 문서 로더, 벡터DB 연결, 유틸 모음이다. RAG 구현에 필수적인 부품들이 들어 있다
- --upgrade : 최신 버전으로 업데이트
- --quiet : 설치 로그 조용히(출력 최소화)
- %pip : 주피터 노트북용 pip

In [2]:
%pip install --upgrade --quiet docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


# 1. 문서 읽기
- Docx2txtLoader : .docx 파일을 텍스트로 읽어주는 도구
# 2. 문서 쪼개기
- RecursiveCharacterTextSplitter : 긴 글을 적당한 길이로 쪼개는 도구


In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 현재 폴더에 있는 tax.docx 파일을 읽을 준비를 함
loader = Docx2txtLoader('./tax.docx')

# tax.docx 전체 내용을 텍스트로 읽고 방금 설정한 규칙으로 여러 조각으로 나눔
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(document_list)

225

# 3-1. 임베딩 모델 준비

In [22]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

# 임베딩 객체 생성
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 3-2. 임베딩된 문서들을 저장

In [45]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma") 처음 한번만 실행(문서 저장용)
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 4. 유사도 검색 테스트

In [46]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)

# 5-1. LLM 준비

In [47]:
from langchain_openai import ChatOpenAI

# GPT-4o 모델 로드
llm=ChatOpenAI(model='gpt-4o')

# 5-2. RAG 전용 프롬프트 준비

In [48]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

# 6. RetrievalQA 체인 구성
- 검색기 + GPT + 프롬프트를 하나의 파이프라인으로 결합
- 이 객체 하나로 RAG 전체가 돌아감

In [49]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# 7. 질문 실행
- 실제로 RAG 사용

In [50]:
ai_message = qa_chain({"query": query})

# 8. 결과 확인

In [51]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '해당 컨텍스트에는 소득세 계산을 위한 구체적인 세율이나 계산 방법이 제시되어 있지 않습니다. 기본세율로 과세되는 근로소득에 대한 언급은 있지만, 연봉 5천만 원에 해당하는 정확한 소득세 금액은 제공된 정보로는 계산할 수 없습니다. 정확한 소득세를 계산하려면 추가적인 세율 정보와 공제를 고려해야 합니다.'}